# Predicción de Suscripción a Depósito a Plazo Fijo - Campaña Telefónica

### Introduccion
En el contexto de la gestión de clientes y la optimización de campañas de marketing, los bancos buscan estrategias más efectivas para aumentar la contratación de sus productos financieros. Este proyecto se centra en el desarrollo de un modelo de machine learning capaz de predecir la probabilidad de que un cliente acepte un depósito a plazo fijo durante la campaña anual telefónica del banco. Utilizando los datos recopilados de los últimos tres años, la solución propuesta permitirá priorizar los clientes con mayor propensión a contratar el producto, optimizando recursos y mejorando los resultados de la campaña de manera eficiente.

### Objetivo
Desarrollar un modelo predictivo de Machine Learning que identifique con precisión qué clientes tienen mayor probabilidad de suscribir un depósito a plazo fijo durante la próxima campaña telefónica, permitiendo una asignación eficiente de recursos y maximizando el retorno de inversión, asegurando un AUC-ROC > 0.75 en validación.


# Analisis y procesamiento de Datos


## Importacion de librerias


In [7]:
# Importamos las librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date


# Scikit-learn

import sklearn.preprocessing as prep
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error

from joblib import dump

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


## Cargar datos

In [11]:
data_test = pd.read_csv("../data/dataset_marketing_test.csv")
data_train = pd.read_csv("../data/dataset_marketing_train.csv")
print(f"\n Dimensiones del conjunto de entrenamiento: {data_train.shape}")
print(f" Dimensiones del conjunto de prueba: {data_test.shape}")



 Dimensiones del conjunto de entrenamiento: (32652, 17)
 Dimensiones del conjunto de prueba: (8536, 17)


## 1. Explorar datos iniciales
Informacion General de los datos

El primer paso es entender la estructura de nuestros datos. Necesitamos saber tamaño , typo de datos , registros ausentes o duplicados 

In [13]:
print("============== Primeras 5 filas ================")
print(data_train.head()) 
print("============= Últimas 5 filas ====================")
print(data_test.tail())

============== Primeras 5 filas ================
   edad       empleo   estado            educacion   impago hipoteca prestamo tipo_contacto  mes dia_semana  contactos_actual  contactos_anterior resultado_anterior  tasa_var_empleo_3m  euribor_3m  ipc_1m target
0    26   management   single    university.degree       no       no       no     telephone  jun        mon                 3                   0        nonexistent                 1.4       4.865  94.465     no
1    42   technician  married  professional.course  unknown       no       no     telephone  jun        mon                 9                   0        nonexistent                 1.4       4.865  94.465     no
2    30       admin.  married    university.degree       no       no       no     telephone  jun        mon                 2                   0        nonexistent                 1.4       4.865  94.465     no
3    55   technician  married             basic.9y       no       no       no     telephone  jun       

In [15]:
print("======== Informacion general del Dataframe =========")
data_train.info()

======== Informacion general del Dataframe =========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32652 entries, 0 to 32651
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   edad                32652 non-null  int64  
 1   empleo              32652 non-null  object 
 2   estado              32652 non-null  object 
 3   educacion           32652 non-null  object 
 4   impago              32652 non-null  object 
 5   hipoteca            32652 non-null  object 
 6   prestamo            32652 non-null  object 
 7   tipo_contacto       32652 non-null  object 
 8   mes                 32652 non-null  object 
 9   dia_semana          32652 non-null  object 
 10  contactos_actual    32652 non-null  int64  
 11  contactos_anterior  32652 non-null  int64  
 12  resultado_anterior  32652 non-null  object 
 13  tasa_var_empleo_3m  32652 non-null  float64
 14  euribor_3m          32652 non-null  float64
 15  

In [20]:
print("=========== Resumen estadistico de las variables de entrenamiento ======== \n")
data_train.describe(include="all")

=========== Resumen estadistico de las variables de entrenamiento ======== 



,edad,empleo,estado,educacion,impago,hipoteca,prestamo,tipo_contacto,mes,dia_semana,contactos_actual,contactos_anterior,resultado_anterior,tasa_var_empleo_3m,euribor_3m,ipc_1m,target
count,32652.000000,32652,32652,32652,32652,32652,32652,32652,32652,32652,32652.000000,32652.000000,32652,32652.000000,32652.000000,32652.000000,32652
unique,NaN,12,4,8,3,3,3,2,10,5,NaN,NaN,3,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,jul,thu,NaN,NaN,nonexistent,NaN,NaN,NaN,no
freq,NaN,8575,19383,10240,26520,17584,26897,25552,6863,7146,NaN,NaN,27474,NaN,NaN,NaN,28655
mean,39.930081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.595461,0.193281,NaN,-0.117987,3.393421,93.463533,NaN
std,10.592293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.819542,0.500932,NaN,1.658776,1.787415,0.601256,NaN
min,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,-3.400000,0.634000,92.201000,NaN
25%,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,-1.800000,1.313000,92.893000,NaN
50%,38.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,0.000000,NaN,-0.100000,4.474000,93.444000,NaN
75%,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,0.000000,NaN,1.400000,4.962000,93.918000,NaN


In [21]:
print("=========== Valores nulos en TRAIN ==========")
print(data_train.isnull().sum())

print("\n=========== Valores nulos en TEST ==========")
print(data_test.isnull().sum())

=========== Valores nulos en TRAIN ==========
edad                  0
empleo                0
estado                0
educacion             0
impago                0
hipoteca              0
prestamo              0
tipo_contacto         0
mes                   0
dia_semana            0
contactos_actual      0
contactos_anterior    0
resultado_anterior    0
tasa_var_empleo_3m    0
euribor_3m            0
ipc_1m                0
target                0
dtype: int64

=========== Valores nulos en TEST ==========
edad                  0
empleo                0
estado                0
educacion             0
impago                0
hipoteca              0
prestamo              0
tipo_contacto         0
mes                   0
dia_semana            0
contactos_actual      0
contactos_anterior    0
resultado_anterior    0
tasa_var_empleo_3m    0
euribor_3m            0
ipc_1m                0
target                0
dtype: int64


In [23]:
print(f'Número de valores duplicados train: {data_train.duplicated().sum()}')
print(f'Número de valores duplicados test: {data_test.duplicated().sum()}')


Número de valores duplicados train: 1553
Número de valores duplicados test: 242
